# 【开源实习】针对任务类型Text Ranking，开发可在香橙派AIpro开发板运行的应用
任务编号：#ICJ8RF
任务链接：[【开源实习】针对任务类型Text Ranking，开发可在香橙派AIpro开发板运行的应用](https://gitee.com/mindspore/community/issues/ICJ8RF)  


## 环境准备
开发者拿到香橙派开发板后，首先需要进行硬件资源确认，镜像烧录及CANN和MindSpore版本的升级，才可运行该案例，具体如下：

开发板：香橙派Aipro或其他同硬件开发板  
开发板镜像: Ubuntu镜像  
CANN Toolkit/Kernels：8.0.0.beta1  
MindSpore: 2.6.0  
MindSpore NLP: 0.4.1  
Python: 3.9

### 镜像烧录
运行该案例需要烧录香橙派官网ubuntu镜像，烧录流程参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--镜像烧录](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html) 章节。

### CANN升级
CANN升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--CANN升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。

### MindSpore升级
MindSpore升级参考[昇思MindSpore官网--香橙派开发专区--环境搭建指南--MindSpore升级](https://www.mindspore.cn/tutorials/zh-CN/r2.7.0rc1/orange_pi/environment_setup.html)章节。


In [1]:
# 安装必要的库
!pip install mindnlp==0.4.1 jieba==0.42.1 sympy==1.14.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
# 导入必要的库
import time
import re
import os
import numpy as np
from mindspore import context
from mindspore import Tensor
from mindnlp.transformers import BertTokenizer, BertModel
from docx import Document

/usr/local/miniconda3/envs/text/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/envs/text/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/miniconda3/envs/text/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/miniconda3/envs/text/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the default dictionary ...
Loading model fro

# 加载模型和分词器

In [3]:

start_time = time.time()
tokenizer = BertTokenizer.from_pretrained('shibing624/text2vec-base-chinese')
model = BertModel.from_pretrained('shibing624/text2vec-base-chinese')
model.set_train(False)  # 设置为评估模式
load_time = time.time() - start_time
print(f"模型加载完成! 耗时: {load_time:.2f}秒")

# 参数设置
TOP_K = 3
DAMPING = 0.85
MAX_ITER = 100
SIM_THRESHOLD = 0.5
MAX_TEXT_LENGTH = 5000

  0%|          | 0.00/319 [00:00<?, ?B/s]

0.00B [00:00, ?B/s]

  0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/miniconda3/envs/text/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


  0%|          | 0.00/434 [00:00<?, ?B/s]

  0%|          | 0.00/390M [00:00<?, ?B/s]

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
模型加载完成! 耗时: 52.52秒


# 加载Text Rank方法

In [4]:

def get_sentence_embedding(sentence):
    """获取句子嵌入向量"""
    inputs = tokenizer(sentence, 
                      max_length=128, 
                      padding='max_length', 
                      return_tensors='ms')
    outputs = model(**inputs)
    return outputs['pooler_output'].asnumpy().squeeze()

def cosine_similarity(a, b):
    """计算余弦相似度"""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-8)

def build_similarity_matrix(sentences):
    """构建句子相似度矩阵"""
    print(f"处理 {len(sentences)} 个句子...")
    start_time = time.time()
    embeddings = []
    
    batch_size = 10
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i+batch_size]
        batch_embeddings = []
        
        for sent in batch:
            batch_embeddings.append(get_sentence_embedding(sent))
            
        embeddings.extend(batch_embeddings)
        
    n = len(sentences)
    matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i != j:
                sim = cosine_similarity(embeddings[i], embeddings[j])
                if sim > SIM_THRESHOLD:
                    matrix[i][j] = sim
                
        if i % 10 == 0 or i == n-1:
            print(f"进度: {i+1}/{n}")
            
    build_time = time.time() - start_time
    return matrix

def textrank(matrix):
    """TextRank算法实现"""
    start_time = time.time()
    
    d = DAMPING
    n = matrix.shape[0]
    weights = np.ones(n) / n  
    
    row_sums = matrix.sum(axis=1)
    norm_matrix = matrix / (row_sums[:, np.newaxis] + 1e-8)
    
    for iter in range(MAX_ITER):
        new_weights = (1 - d) + d * np.dot(norm_matrix.T, weights)
        diff = np.linalg.norm(new_weights - weights)
        
        if diff < 1e-5:
            break
        weights = new_weights

   
    algo_time = time.time() - start_time
    return weights

def preprocess_text(text):
    """预处理文本：清理、截断等"""
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 截断过长的文本
    if len(text) > MAX_TEXT_LENGTH:
        print(f"警告: 文本过长({len(text)}字符)，将截取前{MAX_TEXT_LENGTH}字符")
        text = text[:MAX_TEXT_LENGTH]
        
    return text

def summarize(text, top_k=TOP_K):
    """生成rank评分前top_k个"""
    total_start = time.time()
    
    text = preprocess_text(text)
    
    sentences = re.split(r'[。！？；\n]', text)
    sentences = [s.strip() for s in sentences if len(s.strip()) > 5]
    
    print(f"原始文本句子数: {len(sentences)}")
    
    if len(sentences) <= 1:
        return "文本过短"
    

    sim_matrix = build_similarity_matrix(sentences)
    scores = textrank(sim_matrix)
    
    top_indices = scores.argsort()[-top_k:][::-1]
    top_indices.sort()
    
    # 构建top_k输出
    summary = ''.join([sentences[i] + '。' for i in top_indices])
    
    total_time = time.time() - total_start
    
    # 打印每个句子的详细TextRank情况
    print("\n" + "="*80)
    print("句子TextRank评分详情:")
    for i, (sentence, score) in enumerate(zip(sentences, scores)):
        # 对长句子进行截断，以便在控制台显示
        display_sentence = sentence
        if len(display_sentence) > 60:
            display_sentence = display_sentence[:57] + "..."
        
        # 标记被选中的句子
        mark = " ★" if i in top_indices else ""
        
        print(f"句子 {i+1} (得分: {score:.6f}{mark}):")
        print(f"内容: {display_sentence}")
        print("-"*80)
    
    return summary


# 加载文件处理方法

In [5]:

def read_txt(file_path):
    """读取TXT文件"""
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read()

def read_docx(file_path):
    """读取DOCX文件"""
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


# 加载主函数

In [6]:

def main(top_k=TOP_K):
    """处理输入的文件"""
    print("="*80)
    print("说明:")
    print("- 支持TXT和DOCX格式文件")
    print("- 输入文件路径进行处理")
    print("- 输入'exit'退出程序")
    print("="*80)
    
    while True:
        file_path = input("\n请输入文本文件路径(TXT或DOCX格式, 输入'exit'退出): ").strip()
        
        if file_path.lower() == 'exit':
            print("程序已退出。")
            break
            
        if not os.path.exists(file_path):
            print(f"文件不存在: {file_path}")
            continue
            
        file_ext = os.path.splitext(file_path)[1].lower()
        
        if file_ext not in ['.txt', '.docx']:
            print("不支持的文件格式! 仅支持TXT和DOCX文件。")
            continue
            
        print(f"\n处理文件: {file_path}")
        
        # 读取文件内容
        try:
            if file_ext == '.txt':
                text = read_txt(file_path)
            else:  # .docx
                text = read_docx(file_path)
            
            print(f"读取文本长度: {len(text)} 字符")
        
            summary = summarize(text, top_k)
            
            # 显示结果
            print("\n" + "="*80)
            print("text ranking top3 结果:")
            print("="*80)
            print(summary)
            print("="*80)
            
            
        except Exception as e:
            print(f"处理文件时出错: {str(e)}")



# 运行程序

In [7]:
main(3)

说明:
- 支持TXT和DOCX格式文件
- 输入文件路径进行处理
- 输入'exit'退出程序

处理文件: /opt/image2text/shixi/orange-pi-mindspore/Online/community/18-Text-Ranking/test_01.txt
读取文本长度: 144 字符
原始文本句子数: 9
处理 9 个句子...
进度: 1/9
进度: 9/9

句子TextRank评分详情:
句子 1 (得分: 0.964721):
内容: 水陆草木之花，可爱者甚蕃
--------------------------------------------------------------------------------
句子 2 (得分: 0.985499):
内容: 晋陶渊明独爱菊
--------------------------------------------------------------------------------
句子 3 (得分: 1.006646):
内容: 自李唐来，世人甚爱牡丹
--------------------------------------------------------------------------------
句子 4 (得分: 1.007241):
内容: 予独爱莲之出淤泥而不染，濯清涟而不妖，中通外直，不蔓不枝，香远益清，亭亭净植，可远观而不可亵玩焉
--------------------------------------------------------------------------------
句子 5 (得分: 1.007288):
内容: 予谓菊，花之隐逸者也
--------------------------------------------------------------------------------
句子 6 (得分: 1.008366 ★):
内容: 牡丹，花之富贵者也
--------------------------------------------------------------------------------
句子 7 (得分: 1.021729 ★):
内容: 莲，花之君子者也
---------